In [1]:
# h/t https://www.dremio.com/blog/getting-hands-on-with-polaris-oss-apache-iceberg-and-apache-spark/
import pyspark
from pyspark.sql import SparkSession
import os

## DEFINE SENSITIVE VARIABLES
POLARIS_URI = 'http://polaris:8181/api/catalog'
POLARIS_CATALOG_NAME = 'polariscatalog'
POLARIS_CREDENTIALS = 'root:secret'
POLARIS_SCOPE = 'PRINCIPAL_ROLE:ALL'

In [2]:
conf = (
    pyspark.SparkConf()
        .setAppName('rmoff')
  		#packages
        .set('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.2,org.apache.hadoop:hadoop-aws:3.4.0')
  		#SQL Extensions
        .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions')
  		#Configuring Catalog
        .set('spark.sql.catalog.polaris', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.polaris.warehouse', POLARIS_CATALOG_NAME)
        .set('spark.sql.catalog.polaris.header.X-Iceberg-Access-Delegation', 'true')
        .set('spark.sql.catalog.polaris.catalog-impl', 'org.apache.iceberg.rest.RESTCatalog')
        .set('spark.sql.catalog.polaris.uri', POLARIS_URI)
        .set('spark.sql.catalog.polaris.credential', POLARIS_CREDENTIALS)
        .set('spark.sql.catalog.polaris.scope', POLARIS_SCOPE)
        .set('spark.sql.catalog.polaris.token-refresh-enabled', 'true')
        .set('spark.sql.defaultCatalog', 'polaris') 
)

## Start Spark Session
spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("Spark Running")

Spark Running


25/07/11 09:40:36 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
# # Run this for debug of the config
# for item in spark.sparkContext.getConf().getAll():
#     print(f"{item[0]}: {item[1]}")

Expect the following warning the first time that you run something:

```
WARN RESTSessionCatalog: Iceberg REST client is missing the OAuth2 server URI configuration[…]
```

In [6]:
spark.sql("CREATE NAMESPACE IF NOT EXISTS polaris.db")
spark.sql("SHOW DATABASES IN polaris").show()

+---------+
|namespace|
+---------+
|       db|
|    rmoff|
+---------+



In [36]:
%%sql
USE polaris.rmoff;

++
||
++
++

In [42]:
%%sql
    CREATE TABLE IF NOT EXISTS customers (
  customer_id BIGINT,
  first_name VARCHAR(255),
  last_name VARCHAR(255),
  email VARCHAR(255)
);

++
||
++
++

In [44]:
%%sql

INSERT INTO customers (customer_id, first_name, last_name, email) 
VALUES (1, 'Rey', 'Skywalker', 'rey@rebelscum.org'),
       (2, 'Hermione', 'Granger', 'hermione@hogwarts.edu'),
       (3, 'Tony', 'Stark', 'tony@starkindustries.com');

++
||
++
++

In [45]:
%%sql

SELECT * FROM customers;

customer_id,first_name,last_name,email
1,Rey,Skywalker,rey@rebelscum.org
1,Rey,Skywalker,rey@rebelscum.org
2,Hermione,Granger,hermione@hogwarts.edu
2,Hermione,Granger,hermione@hogwarts.edu
3,Tony,Stark,tony@starkindustries.com
3,Tony,Stark,tony@starkindustries.com
